# Data Preprocessing

In [2]:
# import packages
import pandas as pd
import re 
import unidecode
import string
import itertools
import contractions
import liwc
import gensim
import gsdmm
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nrclex import NRCLex
from collections import Counter
from gsdmm import MovieGroupProcess
from gensim.models.phrases import Phraser, Phrases

In [9]:
# import normal csv
normal_df = pd.read_csv (r'C://Users//lvlip//Documents//BCSI Sem 6//FYP 4202//CSV//normal_twint//NormalCombine.csv', engine='python')
normal_df.head()

,id,conversation_id,created_at,datetime,timezone,user_id,username,name,place,tweet,...,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,target
0,1.470000e+18,1.470000e+18,2021-12-16 14:27:14 Malay Peninsula Standard Time,16/12/2021 14:27,800,2.676289e+09,robertevans97,Robert Evans,NaN,@arielhelwani @DustinPoirier @NateDiaz209 Big ...,...,NaN,NaN,NaN,NaN,"[{'screen_name': 'arielhelwani', 'name': 'Arie...",NaN,NaN,NaN,NaN,normal
1,1.470000e+18,1.470000e+18,2021-12-16 14:27:14 Malay Peninsula Standard Time,16/12/2021 14:27,800,7.190000e+17,flintmorris,FlintMorris,NaN,@ItsRyanGonzales @100Thieves @RabidDoh @thundo...,...,NaN,NaN,NaN,NaN,"[{'screen_name': 'ItsRyanGonzales', 'name': '1...",NaN,NaN,NaN,NaN,normal
2,1.470000e+18,1.470000e+18,2021-12-16 14:27:14 Malay Peninsula Standard Time,16/12/2021 14:27,800,1.370000e+18,chardinite,Chardinite,NaN,@Tinkzorg Gen Z can't wield the mandate of hea...,...,NaN,NaN,NaN,NaN,"[{'screen_name': 'Tinkzorg', 'name': 'Miriam G...",NaN,NaN,NaN,NaN,normal
3,1.470000e+18,1.470000e+18,2021-12-16 14:27:14 Malay Peninsula Standard Time,16/12/2021 14:27,800,1.518807e+08,ghobubo,Blixy Scrumpfer,NaN,@JortsTheCat has his own account and 20K follo...,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,normal
4,1.470000e+18,1.470000e+18,2021-12-16 14:27:14 Malay Peninsula Standard Time,16/12/2021 14:27,800,1.430000e+18,mdarahimkhan2,moni,NaN,@BitKeepOS @NEARProtocol This is a really grea...,...,NaN,NaN,NaN,NaN,"[{'screen_name': 'BitKeepOS', 'name': 'BitKeep...",NaN,NaN,NaN,NaN,normal


In [8]:
# import depressed csv
depressed_df = pd.read_csv (r'C://Users//lvlip//Documents//BCSI Sem 6//FYP 4202//CSV//depression_twint//DepressedCombine.csv', engine='python')
depressed_df.head()

,id,conversation_id,created_at,datetime,timezone,user_id,username,name,place,tweet,...,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,target
0,1.470000e+18,1.470000e+18,2021-12-23 06:47:54 Malay Peninsula Standard Time,23/12/2021 6:47,800,361381402.0,mike_raines48,Michael,NaN,I’m just having the worst week like nothing go...,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,depressed
1,1.470000e+18,1.470000e+18,2021-12-22 06:11:57 Malay Peninsula Standard Time,22/12/2021 6:11,800,361381402.0,mike_raines48,Michael,NaN,The unidentifed neurodivergent urge to figure ...,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,depressed
2,1.470000e+18,1.470000e+18,2021-12-20 07:39:55 Malay Peninsula Standard Time,20/12/2021 7:39,800,361381402.0,mike_raines48,Michael,NaN,Hate when mfs lie to my face when I already kn...,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,depressed
3,1.470000e+18,1.470000e+18,2021-12-19 22:13:35 Malay Peninsula Standard Time,19/12/2021 22:13,800,361381402.0,mike_raines48,Michael,NaN,no one cares enough about me to take the initi...,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,depressed
4,1.470000e+18,1.470000e+18,2021-12-18 12:39:55 Malay Peninsula Standard Time,18/12/2021 12:39,800,361381402.0,mike_raines48,Michael,NaN,"I just want somebody to vibe with , go out, an...",...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,depressed


In [10]:
# combine two dataframe
df = pd.concat([normal_df, depressed_df], axis=0)
df.shape

(10746, 36)

In [11]:
# check the info
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10746 entries, 0 to 5551
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               10746 non-null  float64
 1   conversation_id  10746 non-null  float64
 2   created_at       10746 non-null  object 
 3   datetime         10746 non-null  object 
 4   timezone         10746 non-null  object 
 5   user_id          10746 non-null  float64
 6   username         10746 non-null  object 
 7   name             10746 non-null  object 
 8   place            1 non-null      object 
 9   tweet            10746 non-null  object 
 10  language         10745 non-null  object 
 11  mentions         10745 non-null  object 
 12  urls             10745 non-null  object 
 13  photos           10745 non-null  object 
 14  replies_count    10745 non-null  float64
 15  retweets_count   10745 non-null  float64
 16  likes_count      10745 non-null  float64
 17  hashtags     

In [10]:
# extract required columns
def extract(df):
    df = df[['username', 'datetime', 'tweet', 'target']]
    return df

In [11]:
df = extract(df)

In [12]:
# check the total number of null value
df.isnull().sum()

username    0
datetime    0
tweet       0
target      0
dtype: int64

In [13]:
# check duplicate tweets
df[df.duplicated('tweet')]

,username,datetime,tweet,target
1617,vantksooo,16/12/21 14:26:46,"if u can see this tweet, im encouraging u to v...",normal
2169,themosthi904,17/12/21 16:21:55,??,normal
3062,crazyotters11,17/12/21 16:21:42,????,normal
20,toogay_tootired,17/12/2021 16:21,I feel so sick it’s awful. My Drs apparently c...,depressed
1699,fashionkattz,20/12/2021 15:18,And just like that im #depressed tired of life...,depressed
...,...,...,...,...
5412,im_shy_guy,9/12/2021 4:56,Nobody cares until its too late... No they don...,depressed
5522,shoppingblonde,23/12/2021 12:16,#depression Sucks! I am always so hard on myse...,depressed
5538,lunarmeow,19/12/2021 14:10,I can't stand feeling. Where's the numb I need...,depressed
5541,bo00mer,19/12/2021 3:10,"@SickNotWeak @heylandsberg Bad right now, havi...",depressed


In [14]:
# remove duplicate tweets
def duplicate(df):
    # drop duplicate tweets
    df = df.drop_duplicates(subset=['tweet'])
    df = df.reset_index(drop=True)
    return df

In [15]:
df = duplicate(df)

In [16]:
# remove non-english character
def decode(df):
    decode_data = []
    encoded_data = [x.encode('ascii', 'ignore') for x in df['tweet']]
    for data in encoded_data:
        decode_data.append(data.decode())
    df['tweet'] = decode_data
    return df

In [17]:
df = decode(df)

In [18]:
# create a tweet list from dataframe for cleaning
def tweet_list(df):
    tweet = df['tweet'].iloc[:]
    return tweet

In [19]:
tweet = tweet_list(df)

In [20]:
# text cleaning
def clean(tweet):
    # remove links
    tweet = [re.sub(r'https?://\S+|www\.\S+', '', x) for x in tweet]
    # remove emails
    tweet = [re.sub(r'\S*@\S*\s?', '', x) for x in tweet]
    # remove mentions
    tweet = [re.sub(r'@', '', x) for x in tweet]
    # remove hashtags
    tweet = [re.sub(r'#[A-Za-z0-9_]+', '', x) for x in tweet]
    # remove unicode, change quotation marks, ™ to (tm), and so on
    tweet = [unidecode.unidecode(x) for x in tweet]
    # remove specific character / word
    tweet = [re.sub(r'&amp;', '', x) for x in tweet]
    tweet = [re.sub(r'"', '', x) for x in tweet]
    tweet = [re.sub(r'\(tm\)','', x) for x in tweet]
    tweet = [re.sub(r'\(c\)','', x) for x in tweet]
    return tweet

In [21]:
tweet = clean(tweet)

In [22]:
# convert emojis to text
def emojis(tweet):
    # encode html entities - some of them are emojis
    tweet = [re.sub("&gt;",">", x) for x in tweet]
    tweet = [re.sub("&lt;","<", x) for x in tweet]
    # emojis to text
    tweet = [re.sub(";\)","wink", x) for x in tweet]
    tweet = [re.sub(";-\)","wink", x) for x in tweet]
    tweet = [re.sub(";D","wink", x) for x in tweet]
    tweet = [re.sub(":\)","smiley", x) for x in tweet]
    tweet = [re.sub(":>","smiley", x) for x in tweet]
    tweet = [re.sub(":-\)","smiley", x) for x in tweet]
    tweet = [re.sub(":^\)","smiley", x) for x in tweet]
    tweet = [re.sub(":]","smiley", x) for x in tweet]
    tweet = [re.sub("=\)","smiley", x) for x in tweet]
    tweet = [re.sub(":'\)","happy", x) for x in tweet]
    tweet = [re.sub(">v<","happy", x) for x in tweet]
    tweet = [re.sub(":D","grinning", x) for x in tweet]
    tweet = [re.sub("\^\^","joy", x) for x in tweet]
    tweet = [re.sub("\^__\^","joy", x) for x in tweet]
    tweet = [re.sub("xD","laughing", x) for x in tweet]
    tweet = [re.sub("XD","laughing", x) for x in tweet]
    tweet = [re.sub(":p","cheeky", x) for x in tweet]
    tweet = [re.sub(":/","hesitant", x) for x in tweet]
    tweet = [re.sub("<3","love", x) for x in tweet]
    tweet = [re.sub(">_>","devious", x) for x in tweet]
    tweet = [re.sub(">.>","devious", x) for x in tweet]
    tweet = [re.sub(":\(","sad", x) for x in tweet]
    tweet = [re.sub("T__T","crying", x) for x in tweet]
    tweet = [re.sub(";-;","crying", x) for x in tweet]
    tweet = [re.sub("_\(:3","tired", x) for x in tweet]
    return tweet

In [23]:
tweet = emojis(tweet)

In [24]:
# convert text to lowercase        
def lowercase(tweet):
    tweet = [x.lower() for x in tweet]
    return tweet

In [25]:
tweet = lowercase(tweet)

In [26]:
# expand the contractions
def contracts(tweet):
    tweet = [contractions.fix(x, slang=True) for x in tweet]
    return tweet

In [27]:
tweet = contracts(tweet)

In [28]:
# remove punctuations
def punctuations(tweet):
    for character in string.punctuation:
        tweet = [x.replace(character, ' ') for x in tweet]
    return tweet

In [29]:
tweet = punctuations(tweet)

In [30]:
# fix word lengthening
def lengthening(tweet):
    tweet = [''.join(''.join(s)[:2] for _, s in itertools.groupby(x)) for x in tweet]
    return tweet

In [31]:
tweet = lengthening(tweet)

In [32]:
# expand jargons
def jargons(tweet):
    tweet = [re.sub(r'\bppl\b','people', x) for x in tweet]
    tweet = [re.sub(r'\brn\b','right now', x) for x in tweet]
    tweet = [re.sub(r'\bw\b','with', x) for x in tweet]
    tweet = [re.sub(r'\br\b','are', x) for x in tweet]

    jargons = {
    'asap': 'as fast as possible',
    'brb': 'be right back',
    'lol': 'laugh',
    'lolz': 'laugh',
    'lmao': 'laugh',
    'lmaoo': 'laugh',
    'tbh': 'to be honest',
    'abt': 'about',
    'smh': 'somehow',
    'btw': 'by the way',
    'gatchu': 'got you',
    'atm': 'at the moment',
    'txt': 'text',
    'yrs': 'years',
    'fyi': 'for your information',
    'gna': 'gonna',
    'gn': 'good night',
    'yt': 'youtube',
    'vid': 'video',
    'vids': 'videos',
    'pic': 'picture',
    'thru': 'through',
    'tho': 'though',
    'imma': 'i am going to',
    'bff': 'best friend forever',
    'dm': 'direct message',
    'cmon': 'come on',
    'bcz': 'because',
    'bc': 'because',
    'rly': 'really',
    'idk': 'i do not know',
    'ikr': 'i know right',
    'plz': 'please',
    'pls': 'please',
    'bro': 'brother',
    'bruh': 'brother',
    'dunno': 'do not know'
    }

    for i in range(len(tweet)):
        for word in tweet[i].split():
            if word in jargons:
                tweet[i] = tweet[i].replace(word, jargons[word])
    return tweet

In [33]:
tweet = jargons(tweet)

In [34]:
# remove word with numbers -> such as datetime
def numbers(tweet):
    tweet = [re.sub(r'\w+\d\w+', '', x) for x in tweet]
    tweet = [re.sub(r'\d+', '', x) for x in tweet]
    return tweet

In [35]:
tweet = numbers(tweet)

In [36]:
# change 's -> is, delete s as some become s only
def replace(tweet):
    tweet = [re.sub(r'\bs\b','', x) for x in tweet]
    tweet = [re.sub(r'\bid\b','i had', x) for x in tweet]
    return tweet

In [37]:
tweet = replace(tweet)

In [38]:
# remove whitespace
def whitespace(tweet):
    tweet = [re.sub(' +',' ', x) for x in tweet]
    tweet = [x.rstrip().lstrip() for x in tweet]
    return tweet

In [39]:
tweet = whitespace(tweet)

In [40]:
# extract pos tag
pos_tagged = [nltk.pos_tag(nltk.word_tokenize(x)) for x in tweet]

In [41]:
# WordNetLemmatizer with appropriate pos tags
def pos_tagger(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:         
        return None

In [42]:
# replace the original pos tag with simpler naming
wordnet_tagged = [list(map(lambda x: (x[0], pos_tagger(x[1])), x)) for x in pos_tagged]

In [43]:
# lemmatize tweets with the pos tag created
def lemmatize(wordnet_tagged):
    lemmatizer = WordNetLemmatizer()
    lemmatized_tweet = []

    for i in range (len(wordnet_tagged)):
        lemmatized_sentence = []
        for word, tag in wordnet_tagged[i]:
            if tag is None:
                # if there is no available tag, append the token without lemmatize it
                lemmatized_sentence.append(word)
            else:       
                # use the tag to lemmatize the word
                lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
        lemmatized_join = ' '.join(lemmatized_sentence)
        lemmatized_tweet.append(lemmatized_join)
    return lemmatized_tweet

In [44]:
tweet = lemmatize(wordnet_tagged)

In [45]:
# remove stopwords
def remove_stopwords(tweet):
    # reserve 1st 2nd 3rd person pronoun and "no"
    stop_words = stopwords.words('english')
    stop_words = ['what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are','was', 'were', 
                  'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 
                  'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 
                  'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 
                  'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 
                  'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 
                  'other', 'some', 'such', 'nor', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 
                  'will', 'just', 'should', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y',
                 ]

    text_tokens = [word_tokenize(x) for x in tweet]
    
    combine_without_sw = []
    combine_text = []
    
    for text in text_tokens:
        text_without_sw = [word for word in text if word not in stop_words]
        combine_without_sw.append(text_without_sw)
    
    for word in combine_without_sw:
        combine_join = ' '.join(word)
        combine_text.append(combine_join)
    return combine_text

In [46]:
tweet = remove_stopwords(tweet)

In [47]:
tweet = lowercase(tweet)

In [48]:
# combine tweet and df
def combine(tweet, df):
    # convert tweet list to dataframe
    cleaned_tweet = pd.DataFrame(tweet, columns=['cleaned_tweet'])
    # concat df with cleaned_tweet
    df = pd.concat([df, cleaned_tweet], axis=1)
    # reordering the columns
    df = df[['username', 'datetime', 'tweet', 'cleaned_tweet', 'target']]
    return df

In [49]:
df = combine(tweet, df)

In [50]:
# remove tweets with <= 1 words 
def less_words(df):
    # check the tweets with <= words
    count = 0
    delete = []

    for i in range (len(df)):
        if len(df['cleaned_tweet'][i].split()) <= 1:
            print(df['cleaned_tweet'][i])
            delete.append(i)
            count += 1

    print(count)
    print(delete)
    
    # remove empty elements
    df = df.drop(df.index[delete])
    # reset index
    df = df.reset_index(drop=True)
    return df

In [51]:
df = less_words(df)

come
honour
question
cop
member
beautiful
truth
moon
bronze

win




c


best
definitely
g
thankful

end
literally
fk
yahoo


ww

them

hopefully
fuck





left



disrespect


eve

wallpaper
sea
chevalet

accurate
sure
mine


good



moon


buy
selector
awesome





kid
hahahaha
yes
durrhh

proof

first






yes
sweet




baby

fuck





next



sad
literally
realization
glass



leo



confess
you

profit
show
anything
photographer
spirit
floor
life
you
go
option
hell
exhaust
131
[236, 399, 471, 576, 599, 688, 1242, 1262, 1326, 1666, 1670, 1685, 1686, 1732, 1740, 1773, 1787, 1790, 1808, 1833, 1853, 1869, 1879, 1890, 1901, 1923, 1934, 1946, 1955, 1959, 2007, 2043, 2051, 2053, 2054, 2082, 2101, 2119, 2126, 2129, 2130, 2135, 2170, 2172, 2177, 2228, 2243, 2259, 2264, 2277, 2286, 2310, 2322, 2339, 2377, 2380, 2382, 2402, 2404, 2446, 2466, 2475, 2488, 2505, 2511, 2531, 2545, 2546, 2554, 2581, 2584, 2606, 2622, 2625, 2628, 2636, 2637, 2643, 2657, 2680, 2708, 2718, 2747, 2763, 2782, 2784, 2

# Feature Engineering

In [53]:
# get sentiment polarity score with VADER
def vader(df):
    analyzer = SentimentIntensityAnalyzer()
    df['tweet_compound'] = df['tweet'].apply(lambda x:analyzer.polarity_scores(x)['compound'])
    return df

In [54]:
df = vader(df)

In [55]:
# get emotion lexicon with nrclex
def nrclex(df):
    # all emotions affect frequencies in one column
    df['emotions'] = df['cleaned_tweet'].apply(lambda x: NRCLex(x).affect_frequencies)
    # split emotions into respective columns
    df = pd.concat([df.drop(['emotions'], axis=1), df['emotions'].apply(pd.Series)], axis=1)
    return df

In [56]:
df = nrclex(df)

In [57]:
df.head()

,username,datetime,tweet,cleaned_tweet,target,tweet_compound,fear,anger,anticipation,trust,surprise,positive,negative,sadness,disgust,joy
0,robertevans97,16/12/21 14:27:14,@arielhelwani @DustinPoirier @NateDiaz209 Big ...,big fight dustin want prove point nate,normal,-0.3182,0.25,0.25,0.000000,0.000000,0.000000,0.250000,0.25,0.0,0.0,0.000000
1,flintmorris,16/12/21 14:27:14,@ItsRyanGonzales @100Thieves @RabidDoh @thundo...,they choose one best naruto opening song big,normal,0.6369,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000
2,chardinite,16/12/21 14:27:14,@Tinkzorg Gen Z can't wield the mandate of hea...,gen z not wield mandate heaven confirm,normal,0.5106,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000
3,ghobubo,16/12/21 14:27:14,@JortsTheCat has his own account and 20K follo...,his account follower sweet potato,normal,0.4588,0.00,0.00,0.142857,0.428571,0.142857,0.142857,0.00,0.0,0.0,0.142857
4,mdarahimkhan2,16/12/21 14:27:14,@BitKeepOS @NEARProtocol This is a really grea...,really great excellent project thank you oppor...,normal,0.9604,0.00,0.00,0.250000,0.187500,0.062500,0.312500,0.00,0.0,0.0,0.187500


In [58]:
# get liwc with custom dictionary
def count_liwc(df):
    # load the liwc dictionary
    liwcPath = 'C://Users//lvlip//Documents//BCSI Sem 6//FYP 4202//CSV//liwc_dic.dic'
    # parse the dictionary
    parse, category_names = liwc.load_token_parser(liwcPath)
    # tokenize the tweets
    text_tokens = [word_tokenize(x) for x in df['cleaned_tweet']]
    
    # count the frequency of each category
    combine_liwc = []

    for text in text_tokens:
        word_counts = Counter(category for word in text for category in parse(word))
        #print(word_counts.Counter())
        combine_liwc.append(word_counts)
    
    # extract counter dictionary - previous result Counter({}), extract the {}
    combine_dic = []

    for i in range (len(combine_liwc)):
        dic = {}
        for k, v in combine_liwc[i].items():
            dic[k] = v
        #print(dic)
        combine_dic.append(dic)
        
    # convert dictionary into dataframe
    liwc_df = pd.DataFrame(combine_dic)
    
    # replace NaN with 0
    liwc_df = liwc_df.fillna(0)
    
    # concat df with liwc
    df = pd.concat([df, liwc_df], axis=1)
    return df

In [59]:
df = count_liwc(df)

# GSDMM

In [60]:
# topic modeling
def train_topic(df):
    # tokenize tweets into words
    docs = [word_tokenize(x) for x in df['cleaned_tweet']]
    
    # remove stopwords from docs
    docs_cleaned = []
    stop_words = gensim.parsing.preprocessing.STOPWORDS
    
    for doc in docs:
        temp = []
        for word in doc:
            if word not in stop_words:
                temp.append(word)
        #print(temp)
        docs_cleaned.append(temp)

    # create dictionary of all words in docs_cleaned
    dictionary = gensim.corpora.Dictionary(docs_cleaned)
    
    # find the length of dictionary
    dic_length = len(dictionary)

    # initialize GSDMM
    np.random.seed(1000)
    gsdmm = MovieGroupProcess(K=10, alpha=0.1, beta=0.1, n_iters=30)

    # fit GSDMM model
    gsdmm.fit(docs_cleaned, dic_length)
    return gsdmm, docs_cleaned

In [62]:
gsdmm, docs_cleaned = train_topic(df)

In stage 0: transferred 8593 clusters with 10 clusters populated
In stage 1: transferred 6089 clusters with 10 clusters populated
In stage 2: transferred 5059 clusters with 10 clusters populated
In stage 3: transferred 4319 clusters with 10 clusters populated
In stage 4: transferred 3920 clusters with 10 clusters populated
In stage 5: transferred 3647 clusters with 10 clusters populated
In stage 6: transferred 3478 clusters with 10 clusters populated
In stage 7: transferred 3324 clusters with 10 clusters populated
In stage 8: transferred 3200 clusters with 10 clusters populated
In stage 9: transferred 3226 clusters with 10 clusters populated
In stage 10: transferred 3172 clusters with 10 clusters populated
In stage 11: transferred 3148 clusters with 10 clusters populated
In stage 12: transferred 3129 clusters with 10 clusters populated
In stage 13: transferred 3079 clusters with 10 clusters populated
In stage 14: transferred 3078 clusters with 10 clusters populated
In stage 15: transfe

In [63]:
pickle.dump(gsdmm, open('gsdmm.pkl', 'wb'))

In [606]:
# print number of documents per topic
doc_count = np.array(gsdmm.cluster_doc_count)
print('Number of documents per topic:', doc_count)

Number of documents per topic: [ 303  209 5376  850  136 1179  175  684  772  863]


In [607]:
# topics sorted by the number of document they are allocated to
top_topic = doc_count.argsort()[-10:][::-1]
print('Most important topics:', top_topic)

Most important topics: [2 5 9 3 8 7 0 1 6 4]


In [608]:
# get top words per topic
for topic in top_topic:
    sort_dicts = sorted(gsdmm.cluster_word_distribution[topic].items(), key=lambda k: k[1], reverse=True)[:20]
    print("\nCluster %s : %s"%(topic, sort_dicts))


Cluster 2 : [('feel', 1100), ('want', 962), ('like', 962), ('know', 643), ('life', 435), ('time', 422), ('people', 417), ('fuck', 393), ('day', 377), ('think', 365), ('bad', 350), ('hate', 348), ('thing', 334), ('try', 309), ('need', 279), ('good', 271), ('die', 234), ('love', 212), ('hurt', 211), ('tell', 208)]

Cluster 5 : [('like', 131), ('know', 130), ('people', 128), ('want', 98), ('love', 94), ('think', 79), ('time', 68), ('thing', 66), ('way', 65), ('look', 59), ('good', 59), ('work', 58), ('need', 55), ('life', 52), ('friend', 52), ('use', 51), ('try', 47), ('day', 47), ('best', 43), ('help', 40)]

Cluster 9 : [('time', 77), ('like', 73), ('good', 70), ('day', 67), ('year', 66), ('love', 54), ('think', 48), ('best', 48), ('christmas', 46), ('game', 44), ('happy', 42), ('great', 38), ('morning', 37), ('wait', 36), ('new', 36), ('work', 35), ('man', 35), ('laugh', 34), ('play', 34), ('movie', 33)]

Cluster 3 : [('good', 75), ('time', 67), ('team', 65), ('best', 61), ('year', 59)

In [609]:
# extract topic for each tweet
def extract_topic(df, gsdmm, docs_cleaned, top_topic):
    topic = []
    prob = [gsdmm.choose_best_label(x) for x in docs_cleaned]

    threshold = 0.3
    
    # confirm the topic only when the prob is >= 0.3, else assign 10 as other topic
    for data in prob:
        if data[1] >= threshold:
            topic.append(top_topic[data[0]])
        else:
            topic.append(10)
        
    # add the topic column to df
    df['topic'] = topic
    return df

In [610]:
df = extract_topic(df, gsdmm, docs_cleaned, top_topic)

In [611]:
df.head()

,username,datetime,tweet,cleaned_tweet,target,tweet_compound,fear,anger,anticipation,trust,...,primarysupport,troubleconcentrate,worthlessness/guilt,housing,disturbedsleep,occupational,fatigue/lossenergy,weight/appetite,agitation/retardation,topic
0,robertevans97,16/12/21 14:27:14,@arielhelwani @DustinPoirier @NateDiaz209 Big ...,big fight dustin want prove point nate,normal,-0.3182,0.25,0.25,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,flintmorris,16/12/21 14:27:14,@ItsRyanGonzales @100Thieves @RabidDoh @thundo...,they choose one best naruto opening song big,normal,0.6369,0.00,0.00,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
2,chardinite,16/12/21 14:27:14,@Tinkzorg Gen Z can't wield the mandate of hea...,gen z not wield mandate heaven confirm,normal,0.5106,0.00,0.00,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
3,ghobubo,16/12/21 14:27:14,@JortsTheCat has his own account and 20K follo...,his account follower sweet potato,normal,0.4588,0.00,0.00,0.142857,0.428571,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
4,mdarahimkhan2,16/12/21 14:27:14,@BitKeepOS @NEARProtocol This is a really grea...,really great excellent project thank you oppor...,normal,0.9604,0.00,0.00,0.250000,0.187500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [64]:
df.shape

(10547, 29)

In [612]:
df.to_csv('depression_detection.csv', index=False)